LLM-based extraction

In [ ]:
import os
import re
import json
from docling.document_converter import DocumentConverter
import time
import sys
import openai

OPENAI_KEY  = os.getenv("API_KEY")
EMBED_MODEL = "openai_text_embedding_3_small"
CHAT_MODEL  = "anthropic_claude_3_7_sonnet_v1_0"
BASE_URL    = "https://api.marketplace.novo-genai.com/v1"


openai.api_key = OPENAI_KEY
openai_client = openai.OpenAI(api_key=OPENAI_KEY, base_url=BASE_URL)

start_time = time.time()

folder = "data/included_protocols_after_place_revision"
output_path = "llm_extraction.jsonl"

# Section extraction targets and prompts
sections = [
    ("Objectives and endpoints", "Extract and print the full Objectives and endpoints section (not a summary) from the context below. Use only the information from that section of the protocol, nothing else. Do not output any other information, just the text. "),
    ("Statistical considerations (or analysis)", "Extract and print the full Statistical considerations section (not a summary) from the context below. Use only the information from that section of the protocol, nothing else. Do not output any other information, just the text."),
    ("Adverse events (AEs) requiring additional data collection", "Extract and print the full Adverse events (AEs) requiring additional data collection section (not a summary) from the context below. Use only the information from that section of the protocol, nothing else. Do not output any other information, just the text."),
    ("Clinical safety laboratory assessments (or tests)", 
        "Extract and print the full 'Clinical safety laboratory assessments (or tests)' section from the protocol text below. "
        "The detailed information and tables may appear either in the main body of the protocol (usually in section 8 or 9) or in an appendix (for example, Appendix 2). "
        "If the main section refers to an appendix for details, make sure to find and include the detailed appendix content as well. "
        "If the full section is split between the main section and an appendix, combine all relevant information, tables, and lists into a single, coherent section. "
        "Do not summarize or skip anything, extract the section as it appears in the protocol. Do not output any other information, just the text. "
    ),
]

# Regex for cutting before section of Objectives and endpoints
keywords = r"objective|endpoint|estimand|research question"
pattern = re.compile(
    rf"^#+?\s*\d+\s*[\w\s,()]*?({keywords})[\w\s,()]*?$",
    re.IGNORECASE | re.MULTILINE
)

with open(output_path, "w", encoding="utf-8") as fout:
    processed_count = 0
    for filename in os.listdir(folder):
        if not filename.lower().endswith(".pdf"):
            continue
        file_path = os.path.join(folder, filename)
        print(f"Processing: {filename}")

        try:
            doc = DocumentConverter().convert(file_path).document
            md = doc.export_to_markdown()
        except Exception as e:
            print(f"Could not process {filename}: {e}")
            continue

        # Remove front matter before objectives/endpoints/estimands
        matches = list(pattern.finditer(md))
        print("Matches found:")
        for match in matches:
            print(match.group())
        if matches:
            start_pos = max(0, matches[0].start() - 300)  # Include some context before the match
            md = md[start_pos:]

        section_texts = []
        for section, prompt_prefix in sections:
            prompt = (f"{prompt_prefix}\n\nProtocol text:\n\n{md}")

            try:
                res = openai_client.chat.completions.create(
                    model=CHAT_MODEL,
                    messages=[{"role":"user", "content": prompt}],
                    temperature=0,
                )
                result_text = res.choices[0].message.content.strip()
            except Exception as e:
                print(f"Failed LLM call for {section} in {filename}: {e}")
                result_text = ""

            pretty_section = section.split("(")[0].strip().capitalize()
            if "clinical safety" in section.lower():
                section_header = "Clinical laboratory section"
            elif "objectives" in section.lower():
                section_header = "Endpoints Section"
            elif "statistical" in section.lower():
                section_header = "Statistical considerations section"
            elif "adverse events" in section.lower():
                section_header = "Adverse events section"
            else:
                section_header = pretty_section + " Section"
            section_texts.append(f"### {section_header}\n\n{result_text}")

        json_entry = {
            "filename": filename,
            "text": "\n\n".join(section_texts)
        }
        fout.write(json.dumps(json_entry, ensure_ascii=False) + "\n")
        processed_count += 1

   

print("Done! Wrote:", output_path)
print(f"Processed {processed_count} files.")
print(f"Total time: {time.time() - start_time:.2f} seconds")


C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processing: 4300-protocol-version-4.pdf


C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MFRA0106\.cache\huggingface\hub\models--ds4sd--docling-models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\MFRA0106\App

Matches found:
## 4 Objectives and endpoints
Processing: 4303-protocol-version-3.pdf


C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

Matches found:
## 3 Objectives, endpoints and estimands
Processing: _ Protocol  4954 protocol v.1.0.pdf


C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\MFRA0106\AppData\Lo

Matches found:
## 3 Objectives, endpoints and estimands
Processing: _ Protocol Amendment  protocol version 2.0 (1).pdf


C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\MFRA0106\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Matches found:
## 3 Objectives and endpoints
Done! Wrote: llm_extraction.jsonl
Processed 127 files.
Total time: 41540.51 seconds
